# 1. Importing modules and functions

In [2]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from padelpy import from_sdf
import shap
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# 2.Data entry and curation work set

In [3]:
uploaded_file_ws="datasets/HDAC6_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  3083 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [4]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [5]:
len(y_tr)

3083

# 3.Standardization SDF file for work set

In [6]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  3083 molecules


# 4.Data entry and curation test set

In [7]:
uploaded_file_ts="datasets/HDAC6_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  771 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [8]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [9]:
len(y_ts)

771

# 5.Standardization SDF file for test set

In [10]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  771 molecules


## Calculation MACCS Fingerprints for work set

In [11]:
fp_tr = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ws]

In [12]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [13]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [14]:
savetxt('Models/MACCS/x_tr_MACCS.csv', x_tr, delimiter=',')

In [15]:
x_tr.shape

(3083, 167)

## Calculation  MACCS Fingerprint for test set

In [16]:
fp_ts = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ts]

In [17]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [18]:
x_ts = rdkit_numpy_convert(fp_ts)

In [34]:
x_ts.shape

(771, 167)

In [35]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

 ## GradientBoostingRegressor model building and validation

In [23]:
seed = 42

In [24]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [25]:
param_grid = {'learning_rate': [0.02,0.05],
                  'subsample'    : [0.9, 0.5, 0.1],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4, 10]
                 }

In [26]:
m = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [27]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.02, 0.05], 'max_depth': [4, 10],
                         'n_estimators': [100, 500, 1000],
                         'subsample': [0.9, 0.5, 0.1]},
             verbose=1)

In [28]:
best_GBR = m.best_estimator_

In [29]:
m.best_params_

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 1000, 'subsample': 0.9}

In [30]:
y_pred_ws_GBR = best_GBR.predict(x_tr)

In [31]:
R2_WS = round(r2_score(y_tr, y_pred_ws_GBR), 2)
R2_WS

0.85

In [32]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_GBR)), 2)
RMSE_WS

0.56

In [33]:
y_pred_CV_GBR = cross_val_predict(best_GBR, x_tr, y_tr, cv=cv)

In [34]:
y_pred_CV_GBR

array([4.80483859, 6.33949154, 5.93401577, ..., 6.92789574, 7.81602605,
       7.37840087])

In [35]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_GBR), 2)
Q2_CV

0.62

In [36]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_GBR)), 2)
RMSE_CV

0.71

# 9. Prediction for test set's molecules

In [21]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)
len(y_ts)

771

In [22]:
y_pred_GBR = best_GBR.predict(x_ts)

In [23]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.61

In [24]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.71

# save the model to disk

In [62]:
pickle.dump(best_GBR, open('Models/MACCS/HDAC6_GBR_MACCS.pkl', 'wb'))

# load the model from disk

In [20]:
best_GBR = pickle.load(open('Models/MACCS/HDAC6_GBR_MACCS.pkl', 'rb'))

# 10. Y-randomization GradientBoostingRegressor model

In [64]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_GBR, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  2.9min


True score =  0.62 
Y-randomization =  -0.22 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.1min finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [65]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [66]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.236068,4.123106,3.316625,1.414214,5.099020,2.449490,3.464102,2.236068,2.828427,3.872983,...,1.000000,1.414214,2.000000,2.000000,0.000000,1.414214,1.414214,2.828427,1.414214,2.449490
2,3.464102,4.242640,3.464102,1.732051,5.477226,3.872983,3.872983,2.449490,5.099020,4.582576,...,2.828427,2.828427,2.449490,2.000000,0.000000,2.828427,1.732051,3.741657,1.732051,2.449490
3,3.605551,4.472136,3.872983,1.732051,5.567764,3.872983,4.000000,2.449490,5.099020,4.582576,...,2.828427,3.000000,2.449490,2.449490,0.000000,3.000000,2.236068,3.872983,2.236068,2.645751
4,4.000000,4.472136,4.123106,1.732051,5.744563,4.690416,4.000000,2.645751,5.099020,4.690416,...,2.828427,3.162278,3.000000,2.449490,1.000000,3.000000,2.449490,4.242640,2.449490,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.062258,7.937254,8.000000,8.426149,9.380832,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.306623,8.306623,7.937254,8.000000,8.306623,8.246211,8.062258,8.062258,7.937254
3079,8.124039,8.000000,8.062258,8.426149,9.433981,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.366600,8.366600,7.937254,8.000000,8.306623,8.246211,8.062258,8.124039,7.937254
3080,8.185352,8.000000,8.124039,8.426149,9.539392,8.124039,8.185352,7.874008,8.831760,8.000000,...,8.000000,8.426149,8.602325,8.000000,8.124039,8.366600,8.246211,8.124039,8.124039,7.937254
3081,8.246211,8.000000,8.185352,8.944272,9.591663,8.185352,8.185352,7.937254,8.831760,8.426149,...,8.062258,8.426149,8.602325,8.000000,8.306623,8.544003,8.306623,8.185352,8.185352,8.000000


In [67]:
similarity= neighbors_k

In [68]:
Dmean=np.mean(similarity[1,:])

In [69]:
round(Dmean, 2)

1.36

In [70]:
std=np.std(similarity[1,:])

In [71]:
round(std, 2)

1.11

In [72]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.91


In [73]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [74]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,3.464102,3.872983,0.000000,3.605551,2.000000,0.000000,1.414214,1.732051,1.414214,1.414214,...,1.414214,2.000000,1.000000,1.414214,0.000000,3.162278,1.414214,2.000000,1.000000,2.236068
1,4.000000,4.123106,1.732051,4.123106,2.236068,2.828427,1.414214,2.236068,2.236068,1.732051,...,1.732051,2.000000,1.000000,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.449490
2,4.472136,4.123106,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,3.000000,1.732051,...,2.000000,2.000000,2.449490,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.645751
3,4.472136,4.358899,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,4.472136,2.828427,...,2.236068,2.236068,2.645751,2.000000,0.000000,3.872983,2.000000,2.000000,3.464102,2.828427
4,4.582576,4.582576,3.162278,4.472136,2.645751,5.385165,2.828427,2.449490,4.472136,2.828427,...,2.236068,2.449490,2.645751,2.828427,1.000000,3.872983,2.236068,2.000000,3.464102,2.828427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.306623,8.602325,8.426149,8.306623,8.185352,8.660254,7.681146,8.426149,8.306623,8.660254,...,8.426149,7.810250,8.000000,7.937254,8.062258,8.246211,8.366600,8.246211,7.937254,7.937254
3079,8.426149,8.660254,8.485281,8.306623,8.246211,8.660254,7.745967,8.426149,8.366600,8.660254,...,8.426149,7.937254,8.000000,8.000000,8.124039,8.306623,8.366600,8.306623,8.000000,7.937254
3080,8.485281,8.660254,8.485281,8.306623,8.246211,8.660254,7.810250,8.485281,8.426149,8.717798,...,8.426149,7.937254,8.000000,8.062258,8.124039,8.306623,8.426149,8.366600,8.000000,8.000000
3081,8.485281,8.660254,8.544003,8.366600,8.246211,8.660254,7.810250,8.544003,8.426149,8.717798,...,8.485281,7.937254,8.124039,8.062258,8.426149,8.426149,8.485281,8.426149,8.124039,8.000000


In [75]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[3.464 3.873 0.    3.606 2.    0.    1.414 1.732 1.414 1.414 1.    1.
 3.742 1.732 0.    0.    1.    2.    2.449 2.236 2.449 1.414 2.236 1.
 4.472 1.    0.    4.123 1.414 2.449 1.732 2.646 3.317 3.742 1.732 1.414
 2.449 1.732 1.732 1.732 2.646 1.414 2.236 2.646 2.646 2.236 1.414 1.732
 1.414 0.    2.449 1.732 3.606 3.    2.646 0.    1.732 1.414 2.449 1.414
 1.732 1.732 2.449 0.    1.732 4.    2.    1.414 0.    2.449 1.    1.414
 0.    2.646 1.414 1.    1.732 2.449 3.162 1.414 0.    2.236 1.732 1.732
 3.606 1.414 0.    1.    3.606 0.    0.    2.449 1.732 1.732 0.    3.873
 2.    0.    0.    2.646 0.    3.742 0.    1.414 2.449 2.646 1.414 2.236
 2.    1.    1.    1.732 2.449 0.    2.    1.414 2.    0.    2.449 0.
 2.    0.    0.    2.828 2.828 2.828 1.    1.    2.828 0.    1.414 1.
 1.732 1.414 3.606 1.    2.828 1.732 0.    0.    2.236 1.732 1.414 0.
 2.    2.    0.    1.    2.236 1.    2.828 1.414 2.    3.606 2.449 1.
 0.    2.    1.732 0.    1.414 1.414 2.    0.    0.    0.    1.414 3.

In [76]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False False  True False False  True  True  True  True  True  True  True
 False  True  True  True  True False False False False  True False  True
 False  True  True False  True False  True False False False  True  True
 False  True  True  True False  True False False False False  True  True
  True  True False  True False False False  True  True  True False  True
  True  True False  True  True False False  True  True False  True  True
  True False  True  True  True False False  True  True False  True  True
 False  True  True  True False  True  True False  True  True  True False
 False  True  True False  True False  True  True False False  True False
 False  True  True  True False  True False  True False  True False  True
 False  True  True False False False  True  True False  True  True  True
  True  True False  True False  True  True  True False  True  True  True
 False False  True  True False  True False  True False False False  True
  True False  True  True  True  True False  True  T

In [77]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.67


In [78]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  2   5   6   7   8   9  10  11  13  14  15  16  21  23  25  26  28  30
  34  35  37  38  39  41  46  47  48  49  51  55  56  57  59  60  61  63
  64  67  68  70  71  72  74  75  76  79  80  82  83  85  86  87  89  90
  92  93  94  97  98 100 102 103 106 109 110 111 113 115 117 119 121 122
 126 127 129 130 131 132 133 135 137 138 139 141 142 143 146 147 149 151
 155 156 158 159 160 161 163 164 165 166 168 169 170 172 174 175 176 180
 181 184 185 187 190 192 193 195 196 199 200 201 202 203 204 205 206 208
 209 210 211 212 214 219 220 221 222 223 224 225 226 230 231 232 233 234
 235 237 238 239 240 241 242 245 246 248 249 250 251 253 254 255 256 257
 261 262 264 265 266 270 271 273 275 277 278 279 280 281 282 284 286 288
 289 290 292 294 295 296 297 299 300 302 304 305 306 308 309 310 312 313
 314 315 316 317 318 320 321 324 325 327 332 334 335 336 337 340 342 344
 345 346 347 348 349 350 351 353 354 356 359 360 364 366 367 368 369 370
 371 372 37

In [79]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [80]:
y_pred_GBR_ad=list(y_pred_GBR)

In [81]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [82]:
len(y_pred_GBR_ad)

517

In [83]:
y_ts_ad=list(y_ts)

In [84]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [85]:
len(y_ts_ad)

517

In [86]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.63

In [87]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.69

# SVM model building and validation

In [88]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [89]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [90]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [91]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [92]:
svm.best_params_
best_svm = svm.best_estimator_

In [93]:
svm.best_params_

{'C': 10, 'gamma': 0.1}

In [94]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [95]:
R2_WS = round(r2_score(y_tr, y_pred_ws_svm), 2)
R2_WS

0.93

In [96]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_svm)), 2)
RMSE_WS

0.46

In [97]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [98]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.6

In [99]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.72

# 9. Prediction for test set's molecules

In [100]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [101]:
y_pred_svm = best_svm.predict(x_ts)

In [102]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.6

In [103]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.72

save the model to disk

In [104]:
pickle.dump(best_svm, open('Models/MACCS/HDAC6_SVM_MACCS.pkl', 'wb'))

load the model from disk

In [105]:
best_svm = pickle.load(open('Models/MACCS/HDAC6_SVM_MACCS.pkl', 'rb'))

# 10. Y-randomization SVM model

In [106]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_svm, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(3),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   28.7s


True score =  0.603 
Y-randomization =  -0.39 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   54.6s finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [107]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [108]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.236068,4.123106,3.316625,1.414214,5.099020,2.449490,3.464102,2.236068,2.828427,3.872983,...,1.000000,1.414214,2.000000,2.000000,0.000000,1.414214,1.414214,2.828427,1.414214,2.449490
2,3.464102,4.242640,3.464102,1.732051,5.477226,3.872983,3.872983,2.449490,5.099020,4.582576,...,2.828427,2.828427,2.449490,2.000000,0.000000,2.828427,1.732051,3.741657,1.732051,2.449490
3,3.605551,4.472136,3.872983,1.732051,5.567764,3.872983,4.000000,2.449490,5.099020,4.582576,...,2.828427,3.000000,2.449490,2.449490,0.000000,3.000000,2.236068,3.872983,2.236068,2.645751
4,4.000000,4.472136,4.123106,1.732051,5.744563,4.690416,4.000000,2.645751,5.099020,4.690416,...,2.828427,3.162278,3.000000,2.449490,1.000000,3.000000,2.449490,4.242640,2.449490,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.062258,7.937254,8.000000,8.426149,9.380832,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.306623,8.306623,7.937254,8.000000,8.306623,8.246211,8.062258,8.062258,7.937254
3079,8.124039,8.000000,8.062258,8.426149,9.433981,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.366600,8.366600,7.937254,8.000000,8.306623,8.246211,8.062258,8.124039,7.937254
3080,8.185352,8.000000,8.124039,8.426149,9.539392,8.124039,8.185352,7.874008,8.831760,8.000000,...,8.000000,8.426149,8.602325,8.000000,8.124039,8.366600,8.246211,8.124039,8.124039,7.937254
3081,8.246211,8.000000,8.185352,8.944272,9.591663,8.185352,8.185352,7.937254,8.831760,8.426149,...,8.062258,8.426149,8.602325,8.000000,8.306623,8.544003,8.306623,8.185352,8.185352,8.000000


In [109]:
similarity= neighbors_k

In [110]:
Dmean=np.mean(similarity[1,:])

In [111]:
round(Dmean, 2)

1.36

In [112]:
std=np.std(similarity[1,:])

In [113]:
round(std, 2)

1.11

In [114]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.91


In [115]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [116]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,3.464102,3.872983,0.000000,3.605551,2.000000,0.000000,1.414214,1.732051,1.414214,1.414214,...,1.414214,2.000000,1.000000,1.414214,0.000000,3.162278,1.414214,2.000000,1.000000,2.236068
1,4.000000,4.123106,1.732051,4.123106,2.236068,2.828427,1.414214,2.236068,2.236068,1.732051,...,1.732051,2.000000,1.000000,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.449490
2,4.472136,4.123106,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,3.000000,1.732051,...,2.000000,2.000000,2.449490,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.645751
3,4.472136,4.358899,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,4.472136,2.828427,...,2.236068,2.236068,2.645751,2.000000,0.000000,3.872983,2.000000,2.000000,3.464102,2.828427
4,4.582576,4.582576,3.162278,4.472136,2.645751,5.385165,2.828427,2.449490,4.472136,2.828427,...,2.236068,2.449490,2.645751,2.828427,1.000000,3.872983,2.236068,2.000000,3.464102,2.828427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.306623,8.602325,8.426149,8.306623,8.185352,8.660254,7.681146,8.426149,8.306623,8.660254,...,8.426149,7.810250,8.000000,7.937254,8.062258,8.246211,8.366600,8.246211,7.937254,7.937254
3079,8.426149,8.660254,8.485281,8.306623,8.246211,8.660254,7.745967,8.426149,8.366600,8.660254,...,8.426149,7.937254,8.000000,8.000000,8.124039,8.306623,8.366600,8.306623,8.000000,7.937254
3080,8.485281,8.660254,8.485281,8.306623,8.246211,8.660254,7.810250,8.485281,8.426149,8.717798,...,8.426149,7.937254,8.000000,8.062258,8.124039,8.306623,8.426149,8.366600,8.000000,8.000000
3081,8.485281,8.660254,8.544003,8.366600,8.246211,8.660254,7.810250,8.544003,8.426149,8.717798,...,8.485281,7.937254,8.124039,8.062258,8.426149,8.426149,8.485281,8.426149,8.124039,8.000000


In [117]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[3.464 3.873 0.    3.606 2.    0.    1.414 1.732 1.414 1.414 1.    1.
 3.742 1.732 0.    0.    1.    2.    2.449 2.236 2.449 1.414 2.236 1.
 4.472 1.    0.    4.123 1.414 2.449 1.732 2.646 3.317 3.742 1.732 1.414
 2.449 1.732 1.732 1.732 2.646 1.414 2.236 2.646 2.646 2.236 1.414 1.732
 1.414 0.    2.449 1.732 3.606 3.    2.646 0.    1.732 1.414 2.449 1.414
 1.732 1.732 2.449 0.    1.732 4.    2.    1.414 0.    2.449 1.    1.414
 0.    2.646 1.414 1.    1.732 2.449 3.162 1.414 0.    2.236 1.732 1.732
 3.606 1.414 0.    1.    3.606 0.    0.    2.449 1.732 1.732 0.    3.873
 2.    0.    0.    2.646 0.    3.742 0.    1.414 2.449 2.646 1.414 2.236
 2.    1.    1.    1.732 2.449 0.    2.    1.414 2.    0.    2.449 0.
 2.    0.    0.    2.828 2.828 2.828 1.    1.    2.828 0.    1.414 1.
 1.732 1.414 3.606 1.    2.828 1.732 0.    0.    2.236 1.732 1.414 0.
 2.    2.    0.    1.    2.236 1.    2.828 1.414 2.    3.606 2.449 1.
 0.    2.    1.732 0.    1.414 1.414 2.    0.    0.    0.    1.414 3.

In [118]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False False  True False False  True  True  True  True  True  True  True
 False  True  True  True  True False False False False  True False  True
 False  True  True False  True False  True False False False  True  True
 False  True  True  True False  True False False False False  True  True
  True  True False  True False False False  True  True  True False  True
  True  True False  True  True False False  True  True False  True  True
  True False  True  True  True False False  True  True False  True  True
 False  True  True  True False  True  True False  True  True  True False
 False  True  True False  True False  True  True False False  True False
 False  True  True  True False  True False  True False  True False  True
 False  True  True False False False  True  True False  True  True  True
  True  True False  True False  True  True  True False  True  True  True
 False False  True  True False  True False  True False False False  True
  True False  True  True  True  True False  True  T

In [119]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.67


In [120]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  2   5   6   7   8   9  10  11  13  14  15  16  21  23  25  26  28  30
  34  35  37  38  39  41  46  47  48  49  51  55  56  57  59  60  61  63
  64  67  68  70  71  72  74  75  76  79  80  82  83  85  86  87  89  90
  92  93  94  97  98 100 102 103 106 109 110 111 113 115 117 119 121 122
 126 127 129 130 131 132 133 135 137 138 139 141 142 143 146 147 149 151
 155 156 158 159 160 161 163 164 165 166 168 169 170 172 174 175 176 180
 181 184 185 187 190 192 193 195 196 199 200 201 202 203 204 205 206 208
 209 210 211 212 214 219 220 221 222 223 224 225 226 230 231 232 233 234
 235 237 238 239 240 241 242 245 246 248 249 250 251 253 254 255 256 257
 261 262 264 265 266 270 271 273 275 277 278 279 280 281 282 284 286 288
 289 290 292 294 295 296 297 299 300 302 304 305 306 308 309 310 312 313
 314 315 316 317 318 320 321 324 325 327 332 334 335 336 337 340 342 344
 345 346 347 348 349 350 351 353 354 356 359 360 364 366 367 368 369 370
 371 372 37

In [121]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [122]:
y_pred_svm_ad=list(y_pred_svm)

In [123]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [124]:
len(y_pred_svm_ad)

517

In [125]:
y_ts_ad=list(y_ts)

In [126]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [127]:
len(y_ts_ad)

517

In [128]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.64

In [129]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.68

# Multi-layer Perceptron regressor

In [25]:
from sklearn.neural_network import MLPRegressor

In [28]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [41]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [42]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [43]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(400, 300, 200, 100),
                                                (100, 100, 100), (10, 10, 10),
                                                (50,)],
                         'max_iter': [1000, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [44]:
best_MLPR = m.best_estimator_

In [45]:
m.best_params_

{'activation': 'relu',
 'alpha': 5e-05,
 'hidden_layer_sizes': (400, 300, 200, 100),
 'max_iter': 2000,
 'solver': 'sgd'}

In [46]:
y_pred_ws_MLPR = best_MLPR.predict(x_tr)

In [47]:
R2_WS = round(r2_score(y_tr, y_pred_ws_MLPR), 2)
R2_WS

0.91

In [48]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_MLPR)), 2)
RMSE_WS

0.49

In [49]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

In [50]:
y_pred_CV_MLPR

array([4.5473413, 6.270536 , 6.571547 , ..., 6.8544917, 7.8592067,
       6.622988 ], dtype=float32)

In [51]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.53

In [52]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.76

# 9. Prediction for test set's molecules

In [53]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [54]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [55]:
Q2_TS = round(r2_score(y_ts, y_pred_MLPR), 2)
Q2_TS

0.53

In [56]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_MLPR)), 2)
RMSE_TS

0.75

# save the model to disk

In [82]:
pickle.dump(best_MLPR, open('Models/MACCS/HDAC6_MLPR_MACCS.pkl', 'wb'))

# load the model from disk

In [83]:
best_MLPR = pickle.load(open('Models/MACCS/HDAC6_MLPR_MACCS.pkl', 'rb'))

# 10. Y-randomization MLPR

In [58]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_MLPR, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 22.2min


True score =  0.52 
Y-randomization =  -0.48 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 38.0min finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [84]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [85]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.236068,4.123106,3.316625,1.414214,5.099020,2.449490,3.464102,2.236068,2.828427,3.872983,...,1.000000,1.414214,2.000000,2.000000,0.000000,1.414214,1.414214,2.828427,1.414214,2.449490
2,3.464102,4.242640,3.464102,1.732051,5.477226,3.872983,3.872983,2.449490,5.099020,4.582576,...,2.828427,2.828427,2.449490,2.000000,0.000000,2.828427,1.732051,3.741657,1.732051,2.449490
3,3.605551,4.472136,3.872983,1.732051,5.567764,3.872983,4.000000,2.449490,5.099020,4.582576,...,2.828427,3.000000,2.449490,2.449490,0.000000,3.000000,2.236068,3.872983,2.236068,2.645751
4,4.000000,4.472136,4.123106,1.732051,5.744563,4.690416,4.000000,2.645751,5.099020,4.690416,...,2.828427,3.162278,3.000000,2.449490,1.000000,3.000000,2.449490,4.242640,2.449490,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.062258,7.937254,8.000000,8.426149,9.380832,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.306623,8.306623,7.937254,8.000000,8.306623,8.246211,8.062258,8.062258,7.937254
3079,8.124039,8.000000,8.062258,8.426149,9.433981,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.366600,8.366600,7.937254,8.000000,8.306623,8.246211,8.062258,8.124039,7.937254
3080,8.185352,8.000000,8.124039,8.426149,9.539392,8.124039,8.185352,7.874008,8.831760,8.000000,...,8.000000,8.426149,8.602325,8.000000,8.124039,8.366600,8.246211,8.124039,8.124039,7.937254
3081,8.246211,8.000000,8.185352,8.944272,9.591663,8.185352,8.185352,7.937254,8.831760,8.426149,...,8.062258,8.426149,8.602325,8.000000,8.306623,8.544003,8.306623,8.185352,8.185352,8.000000


In [86]:
similarity= neighbors_k

In [87]:
Dmean=np.mean(similarity[1,:])

In [88]:
round(Dmean, 2)

1.36

In [89]:
std=np.std(similarity[1,:])

In [90]:
round(std, 2)

1.11

In [91]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.91


In [92]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [93]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,3.464102,3.872983,0.000000,3.605551,2.000000,0.000000,1.414214,1.732051,1.414214,1.414214,...,1.414214,2.000000,1.000000,1.414214,0.000000,3.162278,1.414214,2.000000,1.000000,2.236068
1,4.000000,4.123106,1.732051,4.123106,2.236068,2.828427,1.414214,2.236068,2.236068,1.732051,...,1.732051,2.000000,1.000000,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.449490
2,4.472136,4.123106,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,3.000000,1.732051,...,2.000000,2.000000,2.449490,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.645751
3,4.472136,4.358899,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,4.472136,2.828427,...,2.236068,2.236068,2.645751,2.000000,0.000000,3.872983,2.000000,2.000000,3.464102,2.828427
4,4.582576,4.582576,3.162278,4.472136,2.645751,5.385165,2.828427,2.449490,4.472136,2.828427,...,2.236068,2.449490,2.645751,2.828427,1.000000,3.872983,2.236068,2.000000,3.464102,2.828427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.306623,8.602325,8.426149,8.306623,8.185352,8.660254,7.681146,8.426149,8.306623,8.660254,...,8.426149,7.810250,8.000000,7.937254,8.062258,8.246211,8.366600,8.246211,7.937254,7.937254
3079,8.426149,8.660254,8.485281,8.306623,8.246211,8.660254,7.745967,8.426149,8.366600,8.660254,...,8.426149,7.937254,8.000000,8.000000,8.124039,8.306623,8.366600,8.306623,8.000000,7.937254
3080,8.485281,8.660254,8.485281,8.306623,8.246211,8.660254,7.810250,8.485281,8.426149,8.717798,...,8.426149,7.937254,8.000000,8.062258,8.124039,8.306623,8.426149,8.366600,8.000000,8.000000
3081,8.485281,8.660254,8.544003,8.366600,8.246211,8.660254,7.810250,8.544003,8.426149,8.717798,...,8.485281,7.937254,8.124039,8.062258,8.426149,8.426149,8.485281,8.426149,8.124039,8.000000


In [94]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[3.464 3.873 0.    3.606 2.    0.    1.414 1.732 1.414 1.414 1.    1.
 3.742 1.732 0.    0.    1.    2.    2.449 2.236 2.449 1.414 2.236 1.
 4.472 1.    0.    4.123 1.414 2.449 1.732 2.646 3.317 3.742 1.732 1.414
 2.449 1.732 1.732 1.732 2.646 1.414 2.236 2.646 2.646 2.236 1.414 1.732
 1.414 0.    2.449 1.732 3.606 3.    2.646 0.    1.732 1.414 2.449 1.414
 1.732 1.732 2.449 0.    1.732 4.    2.    1.414 0.    2.449 1.    1.414
 0.    2.646 1.414 1.    1.732 2.449 3.162 1.414 0.    2.236 1.732 1.732
 3.606 1.414 0.    1.    3.606 0.    0.    2.449 1.732 1.732 0.    3.873
 2.    0.    0.    2.646 0.    3.742 0.    1.414 2.449 2.646 1.414 2.236
 2.    1.    1.    1.732 2.449 0.    2.    1.414 2.    0.    2.449 0.
 2.    0.    0.    2.828 2.828 2.828 1.    1.    2.828 0.    1.414 1.
 1.732 1.414 3.606 1.    2.828 1.732 0.    0.    2.236 1.732 1.414 0.
 2.    2.    0.    1.    2.236 1.    2.828 1.414 2.    3.606 2.449 1.
 0.    2.    1.732 0.    1.414 1.414 2.    0.    0.    0.    1.414 3.

In [95]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False False  True False False  True  True  True  True  True  True  True
 False  True  True  True  True False False False False  True False  True
 False  True  True False  True False  True False False False  True  True
 False  True  True  True False  True False False False False  True  True
  True  True False  True False False False  True  True  True False  True
  True  True False  True  True False False  True  True False  True  True
  True False  True  True  True False False  True  True False  True  True
 False  True  True  True False  True  True False  True  True  True False
 False  True  True False  True False  True  True False False  True False
 False  True  True  True False  True False  True False  True False  True
 False  True  True False False False  True  True False  True  True  True
  True  True False  True False  True  True  True False  True  True  True
 False False  True  True False  True False  True False False False  True
  True False  True  True  True  True False  True  T

In [96]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.67


In [97]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  2   5   6   7   8   9  10  11  13  14  15  16  21  23  25  26  28  30
  34  35  37  38  39  41  46  47  48  49  51  55  56  57  59  60  61  63
  64  67  68  70  71  72  74  75  76  79  80  82  83  85  86  87  89  90
  92  93  94  97  98 100 102 103 106 109 110 111 113 115 117 119 121 122
 126 127 129 130 131 132 133 135 137 138 139 141 142 143 146 147 149 151
 155 156 158 159 160 161 163 164 165 166 168 169 170 172 174 175 176 180
 181 184 185 187 190 192 193 195 196 199 200 201 202 203 204 205 206 208
 209 210 211 212 214 219 220 221 222 223 224 225 226 230 231 232 233 234
 235 237 238 239 240 241 242 245 246 248 249 250 251 253 254 255 256 257
 261 262 264 265 266 270 271 273 275 277 278 279 280 281 282 284 286 288
 289 290 292 294 295 296 297 299 300 302 304 305 306 308 309 310 312 313
 314 315 316 317 318 320 321 324 325 327 332 334 335 336 337 340 342 344
 345 346 347 348 349 350 351 353 354 356 359 360 364 366 367 368 369 370
 371 372 37

In [98]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [99]:
y_pred_MLPR_ad=list(y_pred_MLPR)

In [100]:
y_pred_MLPR_ad[:] = [x for i,x in enumerate(y_pred_MLPR_ad) if i not in out_Ad]

In [101]:
len(y_pred_MLPR_ad)

517

In [102]:
y_ts_ad=list(y_ts)

In [103]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [104]:
len(y_ts_ad)

517

In [105]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_MLPR_ad), 2)
Q2_TS

0.59

In [106]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_MLPR_ad)), 2)
RMSE_TS

0.71

# k-nearest neighbors

In [117]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

In [118]:
m = GridSearchCV(KNeighborsRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [119]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             verbose=1)

In [120]:
best_kNN = m.best_estimator_

In [121]:
m.best_params_

{'n_neighbors': 4}

In [122]:
y_pred_ws_kNN = best_kNN.predict(x_tr)

In [123]:
R2_WS = round(r2_score(y_tr, y_pred_ws_kNN), 2)
R2_WS

0.75

In [124]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_ws_kNN)), 2)
RMSE_WS

0.63

In [125]:
y_pred_CV_kNN = cross_val_predict(best_kNN, x_tr, y_tr, cv=cv)

In [126]:
y_pred_CV_kNN

array([4.2312503, 7.1425   , 5.71     , ..., 6.54     , 7.8425   ,
       7.075    ], dtype=float32)

In [127]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_kNN), 2)
Q2_CV

0.55

In [128]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_kNN)), 2)
RMSE_CV

0.73

# 9. Prediction for test set's molecules

In [129]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [130]:
y_pred_kNN = best_kNN.predict(x_ts)

In [131]:
Q2_TS = round(r2_score(y_ts, y_pred_kNN), 2)
Q2_TS

0.52

In [132]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_kNN)), 2)
RMSE_TS

0.74

# save the model to disk

In [133]:
pickle.dump(best_kNN, open('Models/MACCS/HDAC6_kNN_MACCS.pkl', 'wb'))

# load the model from disk

In [190]:
best_kNN = pickle.load(open('Models/MACCS/HDAC6_kNN_MACCS.pkl', 'rb'))

# 10. Y-randomization MLPR

In [134]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_kNN, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.3s


True score =  0.55 
Y-randomization =  -0.25 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.6s finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [135]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [136]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.236068,4.123106,3.316625,1.414214,5.099020,2.449490,3.464102,2.236068,2.828427,3.872983,...,1.000000,1.414214,2.000000,2.000000,0.000000,1.414214,1.414214,2.828427,1.414214,2.449490
2,3.464102,4.242640,3.464102,1.732051,5.477226,3.872983,3.872983,2.449490,5.099020,4.582576,...,2.828427,2.828427,2.449490,2.000000,0.000000,2.828427,1.732051,3.741657,1.732051,2.449490
3,3.605551,4.472136,3.872983,1.732051,5.567764,3.872983,4.000000,2.449490,5.099020,4.582576,...,2.828427,3.000000,2.449490,2.449490,0.000000,3.000000,2.236068,3.872983,2.236068,2.645751
4,4.000000,4.472136,4.123106,1.732051,5.744563,4.690416,4.000000,2.645751,5.099020,4.690416,...,2.828427,3.162278,3.000000,2.449490,1.000000,3.000000,2.449490,4.242640,2.449490,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.062258,7.937254,8.000000,8.426149,9.380832,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.306623,8.306623,7.937254,8.000000,8.306623,8.246211,8.062258,8.062258,7.937254
3079,8.124039,8.000000,8.062258,8.426149,9.433981,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.366600,8.366600,7.937254,8.000000,8.306623,8.246211,8.062258,8.124039,7.937254
3080,8.185352,8.000000,8.124039,8.426149,9.539392,8.124039,8.185352,7.874008,8.831760,8.000000,...,8.000000,8.426149,8.602325,8.000000,8.124039,8.366600,8.246211,8.124039,8.124039,7.937254
3081,8.246211,8.000000,8.185352,8.944272,9.591663,8.185352,8.185352,7.937254,8.831760,8.426149,...,8.062258,8.426149,8.602325,8.000000,8.306623,8.544003,8.306623,8.185352,8.185352,8.000000


In [137]:
similarity= neighbors_k

In [138]:
Dmean=np.mean(similarity[1,:])

In [139]:
round(Dmean, 2)

1.36

In [140]:
std=np.std(similarity[1,:])

In [141]:
round(std, 2)

1.11

In [142]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.91


In [143]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [144]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,3.464102,3.872983,0.000000,3.605551,2.000000,0.000000,1.414214,1.732051,1.414214,1.414214,...,1.414214,2.000000,1.000000,1.414214,0.000000,3.162278,1.414214,2.000000,1.000000,2.236068
1,4.000000,4.123106,1.732051,4.123106,2.236068,2.828427,1.414214,2.236068,2.236068,1.732051,...,1.732051,2.000000,1.000000,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.449490
2,4.472136,4.123106,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,3.000000,1.732051,...,2.000000,2.000000,2.449490,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.645751
3,4.472136,4.358899,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,4.472136,2.828427,...,2.236068,2.236068,2.645751,2.000000,0.000000,3.872983,2.000000,2.000000,3.464102,2.828427
4,4.582576,4.582576,3.162278,4.472136,2.645751,5.385165,2.828427,2.449490,4.472136,2.828427,...,2.236068,2.449490,2.645751,2.828427,1.000000,3.872983,2.236068,2.000000,3.464102,2.828427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.306623,8.602325,8.426149,8.306623,8.185352,8.660254,7.681146,8.426149,8.306623,8.660254,...,8.426149,7.810250,8.000000,7.937254,8.062258,8.246211,8.366600,8.246211,7.937254,7.937254
3079,8.426149,8.660254,8.485281,8.306623,8.246211,8.660254,7.745967,8.426149,8.366600,8.660254,...,8.426149,7.937254,8.000000,8.000000,8.124039,8.306623,8.366600,8.306623,8.000000,7.937254
3080,8.485281,8.660254,8.485281,8.306623,8.246211,8.660254,7.810250,8.485281,8.426149,8.717798,...,8.426149,7.937254,8.000000,8.062258,8.124039,8.306623,8.426149,8.366600,8.000000,8.000000
3081,8.485281,8.660254,8.544003,8.366600,8.246211,8.660254,7.810250,8.544003,8.426149,8.717798,...,8.485281,7.937254,8.124039,8.062258,8.426149,8.426149,8.485281,8.426149,8.124039,8.000000


In [145]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[3.464 3.873 0.    3.606 2.    0.    1.414 1.732 1.414 1.414 1.    1.
 3.742 1.732 0.    0.    1.    2.    2.449 2.236 2.449 1.414 2.236 1.
 4.472 1.    0.    4.123 1.414 2.449 1.732 2.646 3.317 3.742 1.732 1.414
 2.449 1.732 1.732 1.732 2.646 1.414 2.236 2.646 2.646 2.236 1.414 1.732
 1.414 0.    2.449 1.732 3.606 3.    2.646 0.    1.732 1.414 2.449 1.414
 1.732 1.732 2.449 0.    1.732 4.    2.    1.414 0.    2.449 1.    1.414
 0.    2.646 1.414 1.    1.732 2.449 3.162 1.414 0.    2.236 1.732 1.732
 3.606 1.414 0.    1.    3.606 0.    0.    2.449 1.732 1.732 0.    3.873
 2.    0.    0.    2.646 0.    3.742 0.    1.414 2.449 2.646 1.414 2.236
 2.    1.    1.    1.732 2.449 0.    2.    1.414 2.    0.    2.449 0.
 2.    0.    0.    2.828 2.828 2.828 1.    1.    2.828 0.    1.414 1.
 1.732 1.414 3.606 1.    2.828 1.732 0.    0.    2.236 1.732 1.414 0.
 2.    2.    0.    1.    2.236 1.    2.828 1.414 2.    3.606 2.449 1.
 0.    2.    1.732 0.    1.414 1.414 2.    0.    0.    0.    1.414 3.

In [146]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False False  True False False  True  True  True  True  True  True  True
 False  True  True  True  True False False False False  True False  True
 False  True  True False  True False  True False False False  True  True
 False  True  True  True False  True False False False False  True  True
  True  True False  True False False False  True  True  True False  True
  True  True False  True  True False False  True  True False  True  True
  True False  True  True  True False False  True  True False  True  True
 False  True  True  True False  True  True False  True  True  True False
 False  True  True False  True False  True  True False False  True False
 False  True  True  True False  True False  True False  True False  True
 False  True  True False False False  True  True False  True  True  True
  True  True False  True False  True  True  True False  True  True  True
 False False  True  True False  True False  True False False False  True
  True False  True  True  True  True False  True  T

In [147]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.67


In [148]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  2   5   6   7   8   9  10  11  13  14  15  16  21  23  25  26  28  30
  34  35  37  38  39  41  46  47  48  49  51  55  56  57  59  60  61  63
  64  67  68  70  71  72  74  75  76  79  80  82  83  85  86  87  89  90
  92  93  94  97  98 100 102 103 106 109 110 111 113 115 117 119 121 122
 126 127 129 130 131 132 133 135 137 138 139 141 142 143 146 147 149 151
 155 156 158 159 160 161 163 164 165 166 168 169 170 172 174 175 176 180
 181 184 185 187 190 192 193 195 196 199 200 201 202 203 204 205 206 208
 209 210 211 212 214 219 220 221 222 223 224 225 226 230 231 232 233 234
 235 237 238 239 240 241 242 245 246 248 249 250 251 253 254 255 256 257
 261 262 264 265 266 270 271 273 275 277 278 279 280 281 282 284 286 288
 289 290 292 294 295 296 297 299 300 302 304 305 306 308 309 310 312 313
 314 315 316 317 318 320 321 324 325 327 332 334 335 336 337 340 342 344
 345 346 347 348 349 350 351 353 354 356 359 360 364 366 367 368 369 370
 371 372 37

In [149]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [150]:
y_pred_kNN_ad=list(y_pred_kNN)

In [151]:
y_pred_kNN_ad[:] = [x for i,x in enumerate(y_pred_kNN_ad) if i not in out_Ad]

In [152]:
len(y_pred_kNN_ad)

517

In [153]:
y_ts_ad=list(y_ts)

In [154]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [155]:
len(y_ts_ad)

517

In [156]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_kNN_ad), 2)
Q2_TS

0.58

In [157]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_kNN_ad)), 2)
RMSE_TS

0.71